In [1]:
import sys
sys.path.append("../")
from utils import pkl_load, pkl_dump, text_load

In [26]:
import pandas as pd
from collections import defaultdict
from copy import deepcopy

In [ ]:
"""
ReadMe

We removed all . in the ICD and phewas code
e.g., 150.0 => 1500; 150.01 => 15001; 150 => 150
"""

In [7]:
icd2phewas = dict()
icd92phewas = dict()
icd102phewas = dict()

In [5]:
phewas_9 = "../raw_data/phewas/phecode_icd9_map_unrolled.csv"
phewas_10 = "../raw_data/phewas/Phecode_map_v1_2_icd10_beta.csv"
phewas_10cm = "../raw_data/phewas/Phecode_map_v1_2_icd10cm_beta.csv"

In [6]:
ph9 = pd.read_csv(phewas_9, dtype=str)
ph9.head()

,icd9,phecode
0,001,008
1,001.0,008
2,001.1,008
3,001.9,008
4,002,008


In [9]:
def ph9m(x):
    icd9 = x['icd9'].strip().replace(".", "")
    phe = x['phecode'].strip().replace(".", "")
    icd2phewas[icd9] = phe
    icd92phewas[icd9] = phe
    
xx = ph9.apply(ph9m, axis=1)

In [14]:
# process icd10, we will focus on icd10cm; only when we cannot find in ICD10cm, we will try icd10

ph10 = pd.read_csv(phewas_10, dtype=str)
ph10.head()

,ICD10,PHECODE,Exl. Phecodes,Excl. Phenotypes
0,A00,008,001-009.99,Intestinal infection
1,A00.0,008,001-009.99,Intestinal infection
2,A00.1,008,001-009.99,Intestinal infection
3,A00.9,008,001-009.99,Intestinal infection
4,A01,008,001-009.99,Intestinal infection


In [22]:
temp_10 = dict()

def ph10f(x):
    try:
        icd10 = x['ICD10'].strip().replace(".", "")
        phe = x['PHECODE'].strip().replace(".", "")
        temp_10[icd10] = phe
    except:
        print(x)
    
xx = ph10.apply(ph10f, axis=1)

ICD10               D05.0
PHECODE               NaN
Exl. Phecodes         NaN
Excl. Phenotypes      NaN
Name: 1451, dtype: object
ICD10               E10.5
PHECODE               NaN
Exl. Phecodes         NaN
Excl. Phenotypes      NaN
Name: 1977, dtype: object
ICD10               E11.5
PHECODE               NaN
Exl. Phecodes         NaN
Excl. Phenotypes      NaN
Name: 1988, dtype: object
ICD10               E13.1
PHECODE               NaN
Exl. Phecodes         NaN
Excl. Phenotypes      NaN
Name: 2006, dtype: object
ICD10               E14
PHECODE             NaN
Exl. Phecodes       NaN
Excl. Phenotypes    NaN
Name: 2015, dtype: object
ICD10               E14.0
PHECODE               NaN
Exl. Phecodes         NaN
Excl. Phenotypes      NaN
Name: 2016, dtype: object
ICD10               E14.1
PHECODE               NaN
Exl. Phecodes         NaN
Excl. Phenotypes      NaN
Name: 2017, dtype: object
ICD10               E14.2
PHECODE               NaN
Exl. Phecodes         NaN
Excl. Phenotypes    

Name: 9504, dtype: object


In [21]:
ph10cm = pd.read_csv(phewas_10cm, dtype=str, encoding='latin')
ph10cm.head()

,icd10cm,icd10cm_str,phecode,phecode_str,exclude_range,exclude_name,leaf,rollup
0,S05.22XD,Ocular laceration and rupture with prolapse or...,870.1,Open wound or laceration of eye or eyelid,870-879.99,injuries & poisonings,1,1
1,S01.101S,Unspecified open wound of right eyelid and per...,870.1,Open wound or laceration of eye or eyelid,870-879.99,injuries & poisonings,1,1
2,S01.102S,Unspecified open wound of left eyelid and peri...,870.1,Open wound or laceration of eye or eyelid,870-879.99,injuries & poisonings,1,1
3,S05.22XA,Ocular laceration and rupture with prolapse or...,870.1,Open wound or laceration of eye or eyelid,870-879.99,injuries & poisonings,1,1
4,S01.149S,Puncture wound with foreign body of unspecifie...,870.1,Open wound or laceration of eye or eyelid,870-879.99,injuries & poisonings,1,1


In [23]:
temp_10cm = dict()

def ph10cmf(x):
    try:
        icd10 = x['icd10cm'].strip().replace(".", "")
        phe = x['phecode'].strip().replace(".", "")
        temp_10cm[icd10] = phe
    except:
        print(x)
    
xx = ph10cm.apply(ph10cmf, axis=1)

In [27]:
icd102phewas = deepcopy(temp_10cm)

for k, v in temp_10.items():
    if k not in icd102phewas:
        icd102phewas[k] = v

In [28]:
len(icd102phewas), len(temp_10cm)

(84123, 82991)

In [29]:
for k, v in icd102phewas.items():
    icd2phewas[k] = v

In [30]:
len(icd2phewas), len(icd92phewas), len(icd102phewas)

(97821, 13707, 84123)

In [31]:
pkl_dump(icd2phewas, "../resources/icd2phewas.pkl")
pkl_dump(icd92phewas, "../resources/icd92phewas.pkl")
pkl_dump(icd102phewas, "../resources/icd102phewas.pkl")